#### CSC 180 Intelligent Systems 

#### William Lorence

#### California State University, Sacramento


# Project 2: Modern Low Footprint Cyber Attack Protection
## Reading the Data
The code below reads the data from the dataset and creates dataframes. Values of "-" are treated as N/A and entries with this value are dropped from the dataframe.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

path = "./dataset/"
save_path = "./models/"

#Defines filepaths for the data sets
training_set = os.path.join(path,"UNSW_NB15_training-set.csv")
test_set = os.path.join(path,"UNSW_NB15_test-set.csv")

#Loads files into dataframes
df_training_set = pd.read_csv(training_set, na_values = ['-'])
df_test_set = pd.read_csv(test_set, na_values = ['-'])

#Removes rows with a "-" in any column
df_training_set.dropna(inplace = True)
df_test_set.dropna(inplace = True)

print(len(df_training_set))
print(len(df_test_set))

df_training_set.head()

81173
35179


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
11,12,2.093085,tcp,smtp,FIN,62,28,56329,2212,42.520967,...,1,2,0,0,0,1,1,0,Normal,0
15,16,0.000002,udp,snmp,INT,2,0,138,0,500000.001300,...,1,4,0,0,0,2,1,0,Normal,0
17,18,0.393556,tcp,http,FIN,10,8,860,1096,43.195886,...,1,2,0,0,1,1,3,0,Normal,0
21,22,0.338017,tcp,http,FIN,10,6,998,268,44.376468,...,1,1,0,0,1,2,3,0,Normal,0


## Data Filtering
The code below removes categorical values that are not present in both datasets. As visible via the print statements, roughly 600 entries are dropped from the training set, while only one is dropped from the test set.

In [2]:
#Removes categorical values not present in both datasets
categorical_columns = ['proto', 'service', 'state', 'attack_cat']

for column in categorical_columns:
    unique_values_training = set(df_training_set[column].unique())
    unique_values_test = set(df_test_set[column].unique())

    print(unique_values_training)
    print(unique_values_test)
    
    common_values = unique_values_training.intersection(unique_values_test)
    print(common_values)

    df_training_set = df_training_set[df_training_set[column].isin(common_values)]
    df_test_set = df_test_set[df_test_set[column].isin(common_values)]

print(len(df_training_set))
print(len(df_test_set))

{'udp', 'tcp'}
{'udp', 'tcp'}
{'udp', 'tcp'}
{'dns', 'ftp-data', 'snmp', 'radius', 'smtp', 'pop3', 'dhcp', 'ftp', 'ssl', 'ssh', 'http', 'irc'}
{'dns', 'ftp-data', 'snmp', 'radius', 'smtp', 'pop3', 'dhcp', 'ftp', 'ssl', 'ssh', 'http', 'irc'}
{'dns', 'ftp-data', 'radius', 'snmp', 'smtp', 'pop3', 'dhcp', 'ftp', 'ssl', 'ssh', 'http', 'irc'}
{'RST', 'INT', 'FIN', 'REQ', 'CON'}
{'INT', 'FIN', 'ACC', 'REQ', 'CON'}
{'CON', 'REQ', 'INT', 'FIN'}
{'Exploits', 'Fuzzers', 'Worms', 'Backdoor', 'DoS', 'Normal', 'Generic', 'Reconnaissance', 'Analysis'}
{'Exploits', 'Fuzzers', 'Worms', 'Backdoor', 'DoS', 'Normal', 'Generic', 'Reconnaissance'}
{'Exploits', 'Fuzzers', 'Worms', 'Backdoor', 'DoS', 'Normal', 'Generic', 'Reconnaissance'}
80595
35178


The columns 'id' and 'is_sm_ips_ports' are dropped from the dataframes: 'id' is irrelevent to the data at hand, and 'is_sm_ips_ports' causes errors when calculating z scores (likely because it is always 0).

In [3]:
df_training_set.drop('id', axis = 1, inplace = True)
df_training_set.drop('is_sm_ips_ports', axis = 1, inplace = True)

df_test_set.drop('id', axis = 1, inplace = True)
df_test_set.drop('is_sm_ips_ports', axis = 1, inplace = True)

The numerical data is then normalized via z-score.

In [4]:
#Finds numerical data (columns that are not categorical)
numerical_columns = set(df_training_set.columns.symmetric_difference(categorical_columns))

from scipy.stats import zscore

def z_score_numerical(df, names):
    for name in names:
        df[name] = zscore(df[name])

z_score_numerical(df_training_set, numerical_columns)
z_score_numerical(df_test_set, numerical_columns)

Finally, the next bit of code encodes the now-filtered categorical values into their own columns.

In [5]:
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue) given a dataframe and a list of column names
def encode_text_dummy_loop(df, names):
    for name in names:
        dummies = pd.get_dummies(df[name])
        for x in dummies.columns:
            dummy_name = "{}-{}".format(name, x)
            df[dummy_name] = dummies[x]
        df.drop(name, axis=1, inplace=True)

encode_text_dummy_loop(df_training_set, categorical_columns)
encode_text_dummy_loop(df_test_set, categorical_columns)

df_training_set.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state-INT,state-REQ,attack_cat-Backdoor,attack_cat-DoS,attack_cat-Exploits,attack_cat-Fuzzers,attack_cat-Generic,attack_cat-Normal,attack_cat-Reconnaissance,attack_cat-Worms
3,0.181087,-0.048209,-0.044896,-0.056076,-0.078043,-0.627238,-1.049691,1.573314,-0.613107,-0.217295,...,False,False,False,False,False,False,False,True,False,False
11,0.273769,0.215678,0.075935,0.168768,-0.069786,-0.627070,-1.049691,1.573314,-0.610533,-0.212616,...,False,False,False,False,False,False,False,True,False,False
15,-0.197720,-0.100986,-0.135519,-0.058054,-0.082452,2.277630,0.815996,-0.710229,2.785246,-0.220574,...,True,False,False,False,False,False,False,True,False,False
17,-0.109068,-0.058764,-0.075104,-0.055140,-0.076176,-0.627066,-1.049691,1.573314,-0.612947,-0.201545,...,False,False,False,False,False,False,False,True,False,False
21,-0.121579,-0.058764,-0.090208,-0.054583,-0.080917,-0.627059,0.815996,1.573314,-0.612879,-0.215399,...,False,False,False,False,False,False,False,True,False,False


The data is now ready for processing.